In [82]:
import pandas as pd

def cargar_tablas_mixtas():
    archivos = {
        'DIM_CATEGORY': ('DIM_CATEGORY.csv', 'csv'),
        'DIM_PRODUCT': ('DIM_PRODUCT.xlsx', 'xlsx'),
        'DIM_SEGMENT': ('DIM_SEGMENT.xlsx', 'xlsx'),
        'DIM_CALENDAR': ('DIM_CALENDAR.xlsx', 'xlsx'),
        'FACT_SALES': ('FACT_SALES.csv', 'csv')
    }

    dataframes = {}

    for nombre_tabla, (archivo, formato) in archivos.items():
        if formato == 'csv':
            df = pd.read_csv(archivo)
        elif formato == 'xlsx':
            df = pd.read_excel(archivo)
        else:
            raise ValueError(f"Formato no soportado para la tabla {nombre_tabla}")
        
        dataframes[nombre_tabla] = df

    return dataframes


In [86]:
# Carga los archivos en la misma carpeta del script
dfs = cargar_tablas_mixtas()

# Despliega la tabla
#print(dfs['DIM_PRODUCT'].head())


In [87]:
def revisar_tablas(dataframes):
    for nombre, df in dataframes.items():
        print(f"\n{'='*60}")
        print(f"Revisión de la tabla: {nombre}")
        print(f"{'='*60}\n")

        # Primeras filas
        print("Primeras filas con funcion head():")
        print(df.head(), "\n")

        # Info general
        print("Información general con funcion info():")
        df.info()
        print("\n")

        # Estadísticas
        print("Estadísticas con describe():")
        print(df.describe(include='all'), "\n")


In [88]:
# Cargar los DataFrames con formatos mixtos
dataframes = cargar_tablas_mixtas();

# Revisar la estructura y contenido de cada uno, 
#se deja comentada la línea para ahorrar espacio en pantalla

#revisar_tablas(dataframes);


In [89]:
def limpiar_dataframes(dataframes, formato_fecha="%Y-%m-%d"):
    dataframes_limpios = {}

    for nombre, df in dataframes.items():
        original_shape = df.shape

        # Normaliza nombres de columnas
        df.columns = df.columns.str.strip().str.replace(' ', '_').str.upper()

        # Limpieza de columnas
        for col in df.columns:
            if df[col].dtype == 'object':
                df[col] = df[col].astype(str).str.strip()
                # Intentar convertir a fecha
                try:
                    fechas_convertidas = pd.to_datetime(df[col], format=formato_fecha, errors='coerce')
                    if fechas_convertidas.notna().sum() > 0:
                        df[col] = fechas_convertidas
                except Exception:
                    pass
            else:
                # Convierte columnas clave numéricas a texto si son relevantes
                if col in ['CATEGORY', 'SEGMENT']:
                    df[col] = df[col].astype(str)

        df = df.fillna("SIN_VALOR")
        df = df.drop_duplicates()

        dataframes_limpios[nombre] = df

    return dataframes_limpios



In [98]:
dfs = cargar_tablas_mixtas();
dfs_limpios = limpiar_dataframes(dfs);
print(dfs_limpios)
"""
Los nombres de los DataFrames son:

PRODUCTOS

VENTAS

CATEGORIAS

SEGMENTOS

Las claves de unión son:

ID_PRODUCTO entre VENTAS y PRODUCTOS

ID_CATEGORIA entre PRODUCTOS y CATEGORIAS

ID_SEGMENTO entre PRODUCTOS y SEGMENTOS
"""


{'DIM_CATEGORY':    ID_CATEGORY                    CATEGORY
0            1  FABRIC TREATMENT and SANIT
1            2                    AIR CARE
2            3                LAVAVAJILLAS
3            4            MEGA SUPERFICIES
4            5         LAVATORY CARE & BRC, 'DIM_PRODUCT':     MANUFACTURER     BRAND              ITEM  \
0     INDS. ALEN  CLORALEX     0000075000592   
1     INDS. ALEN  CLORALEX     0000075000608   
2     INDS. ALEN  CLORALEX     0000075000615   
3     INDS. ALEN  CLORALEX     0000075000622   
4     INDS. ALEN  CLORALEX     0000075000639   
..           ...       ...               ...   
500      RECKITT    VANISH  7501058789778BP1   
501   INDS. ALEN  CLORALEX     7501025412142   
502   INDS. ALEN  CLORALEX               nan   
503       CLOROX    CLOROX               nan   
504       CLOROX    CLOROX  7502240948188BP1   

                                      ITEM_DESCRIPTION CATEGORY   FORMAT  \
0    CLORALEX EL RENDIDOR BOT.PLAST. 250ML NAL. 000...  

'\nLos nombres de los DataFrames son:\n\nPRODUCTOS\n\nVENTAS\n\nCATEGORIAS\n\nSEGMENTOS\n\nLas claves de unión son:\n\nID_PRODUCTO entre VENTAS y PRODUCTOS\n\nID_CATEGORIA entre PRODUCTOS y CATEGORIAS\n\nID_SEGMENTO entre PRODUCTOS y SEGMENTOS\n'

In [99]:
# Renombrar los DataFrames
productos = dfs_limpios['DIM_PRODUCT']
ventas = dfs_limpios['FACT_SALES']
categorias = dfs_limpios['DIM_CATEGORY']
segmentos = dfs_limpios['DIM_SEGMENT']

# Unir ventas con productos
ventas_productos = ventas.merge(productos, left_on='ITEM_CODE', right_on='ITEM', how='left')

# Convertir tipos por seguridad
ventas_productos['CATEGORY'] = ventas_productos['CATEGORY'].astype(str)
categorias['CATEGORY'] = categorias['CATEGORY'].astype(str)

# Agregar dimensión de categoría
ventas_productos_categorias = ventas_productos.merge(categorias, on='CATEGORY', how='left')

# Validar columna de segmento
if 'SEGMENT' in ventas_productos_categorias.columns and 'SEGMENT' in segmentos.columns:
    ventas_productos_categorias['SEGMENT'] = ventas_productos_categorias['SEGMENT'].astype(str)
    segmentos['SEGMENT'] = segmentos['SEGMENT'].astype(str)
    tabla_consolidada = ventas_productos_categorias.merge(segmentos, on='SEGMENT', how='left')
else:
    tabla_consolidada = ventas_productos_categorias  # Si no existe SEGMENT, seguimos sin ese merge

# Ordenar por categoría
if 'CATEGORY' in tabla_consolidada.columns:
    tabla_consolidada = tabla_consolidada.sort_values(by='CATEGORY')

# Vista previa del resultado
print(tabla_consolidada.head())

        WEEK         ITEM_CODE  TOTAL_UNIT_SALES  TOTAL_VALUE_SALES  \
0      34-22  7501058792808BP2             0.006              0.139   
81341  22-23     7501058757463             0.002              0.086   
81340  22-23     7501058751461             0.443             80.332   
81339  22-23     7501058716095             0.137             22.541   
81338  22-23     7501058751188             0.728             71.366   

       TOTAL_UNIT_AVG_WEEKLY_SALES              REGION MANUFACTURER   BRAND  \
0                            1.000  TOTAL AUTOS AREA 5      RECKITT  VANISH   
81341                        2.000  TOTAL AUTOS AREA 3      RECKITT  VANISH   
81340                        4.025  TOTAL AUTOS AREA 3      RECKITT  VANISH   
81339                        4.567  TOTAL AUTOS AREA 3      RECKITT  VANISH   
81338                        4.788  TOTAL AUTOS AREA 3      RECKITT  VANISH   

                   ITEM                                   ITEM_DESCRIPTION  \
0      7501058792808

In [102]:
#Despliegue de tabla consolidada final para observar estructura de datos
print("Columnas de la tabla consolidada:")
print(tabla_consolidada.shape)
print(tabla_consolidada.columns.tolist())

Columnas de la tabla consolidada:
(122002, 16)
['WEEK', 'ITEM_CODE', 'TOTAL_UNIT_SALES', 'TOTAL_VALUE_SALES', 'TOTAL_UNIT_AVG_WEEKLY_SALES', 'REGION', 'MANUFACTURER', 'BRAND', 'ITEM', 'ITEM_DESCRIPTION', 'CATEGORY', 'FORMAT', 'ATTR1', 'ATTR2', 'ATTR3', 'ID_CATEGORY']


In [103]:
#Exportando tabla hacia archivo csv para analisis posterior
tabla_consolidada.to_csv("tabla_consolidada_final.csv", index=False)